<h1 align='center'>AFP Ideal</h1>

Considerar que el APV-A tiene un límite de bonificación de 6 UTM anuales. Este beneficio viene dado por el estado y es un 15% del dinero aportado a los ahorros. Revisar si es preferible el régimen A o B de acuerdo al sueldo, buscando los tramos imponibles en https://www.sii.cl/valores_y_fechas/impuesto_2da_categoria/impuesto2023.htm

In [5]:
from bs4 import BeautifulSoup
from datetime import datetime
from io import StringIO
import pandas as pd
import re
import requests



fecha_nacimiento = datetime.strptime('1994-03-01', '%Y-%m-%d')
edad_de_jubilacion = 65

salario_liquido = 2500000
salario_bruto = 2983813
monto_ahorrado = 5018141

In [6]:
url = r'https://www.spensiones.cl/apps/rentabilidad/getRentabilidad.php?tiprent=FP'
r = requests.get(url)

soup = BeautifulSoup(r.text, 'html.parser')

# Create a list of tables looking for class 'table'
tables = soup.find_all('table', {'class': 'table'})

# Dictionary by fund type and name
dict_returns = {'Fondo A': {'CAPITAL': None, 'CUPRUM': None, 'HABITAT': None, 'MODELO': None, 'PLANVITAL': None, 'PROVIDA': None, 'UNO': None, 'SISTEMA': None}, 
                'Fondo B': {'CAPITAL': None, 'CUPRUM': None, 'HABITAT': None, 'MODELO': None, 'PLANVITAL': None, 'PROVIDA': None, 'UNO': None, 'SISTEMA': None}, 
                'Fondo C': {'CAPITAL': None, 'CUPRUM': None, 'HABITAT': None, 'MODELO': None, 'PLANVITAL': None, 'PROVIDA': None, 'UNO': None, 'SISTEMA': None}, 
                'Fondo D': {'CAPITAL': None, 'CUPRUM': None, 'HABITAT': None, 'MODELO': None, 'PLANVITAL': None, 'PROVIDA': None, 'UNO': None, 'SISTEMA': None}, 
                'Fondo E': {'CAPITAL': None, 'CUPRUM': None, 'HABITAT': None, 'MODELO': None, 'PLANVITAL': None, 'PROVIDA': None, 'UNO': None, 'SISTEMA': None}}

# Fill the dictionary with the values from each fund for 'Últimos 12 meses Dic 2022-Nov 2023'
for table in tables[1:]:
    # Convert the HTML string to a StringIO object
    html_string = str(table)
    html_data = StringIO(html_string)
    df = pd.read_html(html_data)[0]

    # Capture the fund type using re from the first column
    match = re.search(r'TIPO ([A-E])', df.columns[0][0])
    fund_type = match.group(1) if match else None

    if fund_type:
        fund_name = f"Fondo {fund_type}"
        # Set the first column as the index
        df.set_index(df.columns[0], inplace=True)

        for afp in df.index:
            if afp in dict_returns[fund_name].keys():
                dict_returns[fund_name][afp] = df.loc[afp, df.columns[2]]
                
df_returns = pd.DataFrame(dict_returns)
df_returns

,Fondo A,Fondo B,Fondo C,Fondo D,Fondo E
CAPITAL,"-3,42%","-2,05%","-2,54%","-3,98%","-4,32%"
CUPRUM,"-3,04%","-1,89%","-2,44%","-3,74%","-4,29%"
HABITAT,"-2,97%","-1,86%","-2,53%","-4,21%","-4,77%"
MODELO,"-2,79%","-1,78%","-2,16%","-3,73%","-4,34%"
PLANVITAL,"-2,92%","-1,69%","-2,39%","-3,93%","-4,55%"
PROVIDA,"-3,66%","-2,49%","-3,33%","-4,47%","-4,86%"
UNO,"-2,90%","-1,99%","-2,44%","-3,81%","-4,38%"
SISTEMA,"-3,18%","-1,99%","-2,67%","-4,16%","-4,54%"


In [7]:
df_returns_enriched = df_returns.copy()

# Convert all values from a string like '1,23%' to a float like 0.0123
for column in df_returns_enriched.columns:
    df_returns_enriched[column] = df_returns_enriched[column].str.replace('%', '')\
        .str.replace(',', '.').astype(float) / 100


In [37]:
comisiones = {'CAPITAL': 0.0144, 'CUPRUM': 0.0144, 'HABITAT': 0.0127, 'MODELO': 0.0058,
              'PLANVITAL': 0.0116, 'PROVIDA': 0.0145, 'UNO': 0.0049}

df_returns_enriched

,Fondo A,Fondo B,Fondo C,Fondo D,Fondo E
CAPITAL,-0.0342,-0.0205,-0.0254,-0.0398,-0.0432
CUPRUM,-0.0304,-0.0189,-0.0244,-0.0374,-0.0429
HABITAT,-0.0297,-0.0186,-0.0253,-0.0421,-0.0477
MODELO,-0.0279,-0.0178,-0.0216,-0.0373,-0.0434
PLANVITAL,-0.0292,-0.0169,-0.0239,-0.0393,-0.0455
PROVIDA,-0.0366,-0.0249,-0.0333,-0.0447,-0.0486
UNO,-0.0290,-0.0199,-0.0244,-0.0381,-0.0438
SISTEMA,-0.0318,-0.0199,-0.0267,-0.0416,-0.0454


In [66]:
dict_calc = {afp: {'Fondo': None, 'Comisión %': None, 'Rentabilidad %': None,
                   'Comisiones': None, 'Rentabilidad': None, 'Saldo': None}
                   for afp in comisiones.keys()}

for afp, calc in dict_calc.items():
    # Find the best results for this AFP
    best_results = max(df_returns_enriched.loc[afp])

    # Find the column for the max value of each row
    best_fund = df_returns_enriched.loc[afp].idxmax()

    calc['Fondo'] = best_fund
    calc['Comisión %'] = comisiones[afp]
    calc['Rentabilidad %'] = best_results

    # Next 3 months projection
    calc['Comisiones'] = (comisiones[afp] * salario_bruto) * 3

    rent_01 = best_results * monto_ahorrado
    rent_02 = best_results * (monto_ahorrado + rent_01)
    rent_03 = best_results * (monto_ahorrado + rent_01 + rent_02)
    calc['Rentabilidad'] = rent_01 + rent_02 + rent_03

    calc['Saldo'] = calc['Rentabilidad'] - calc['Comisiones']

    # Transpose the following dataframe
df_decide = pd.DataFrame(dict_calc)

# Transpose the dataframe
df_decide = df_decide.T

df_decide.sort_values(by='Saldo', ascending=False)

,Fondo,Comisión %,Rentabilidad %,Comisiones,Rentabilidad,Saldo
MODELO,Fondo B,0.0058,-0.0178,51918.3462,-263227.187087,-315145.533287
UNO,Fondo B,0.0049,-0.0199,43862.0511,-293660.861605,-337522.912705
PLANVITAL,Fondo B,0.0116,-0.0169,103836.6924,-250144.276555,-353980.968955
HABITAT,Fondo B,0.0127,-0.0186,113683.2753,-274836.330634,-388519.605934
CUPRUM,Fondo B,0.0144,-0.0189,128900.7216,-279184.88308,-408085.60468
CAPITAL,Fondo B,0.0144,-0.0205,128900.7216,-302332.282146,-431233.003746
PROVIDA,Fondo B,0.0145,-0.0249,129795.8655,-365598.711206,-495394.576706


In [59]:
# Transpose the following dataframe
df_decide = pd.DataFrame(dict_calc)

# Transpose the dataframe
df_decide = df_decide.T

df_decide.to_excel('decide.xlsx')

In [13]:
url = r'https://www.spensiones.cl/portal/institucional/594/w3-article-2810.html'
r = requests.get(url)

soup = BeautifulSoup(r.text, 'html.parser')

# Create a list of tables looking for class 'table'
table = soup.find_all('li')
table

# # Read the first table as a pandas dataframe
# df = pd.read_html(str(table))

[<li class="pnid-581 cid-31"><a href="w3-propertyname-581.html">QuiÃ©nes Somos</a></li>,
 <li class="current pnid-639 cid-31"><a href="w3-propertyname-639.html">EducaciÃ³n Previsional</a></li>,
 <li class="pnid-579 cid-19"><a href="w3-propertyname-579.html">FiscalizaciÃ³n y RegulaciÃ³n</a></li>,
 <li class="pnid-621 cid-25"><a href="w3-propertyname-621.html">EstadÃ­sticas e Informes</a></li>,
 <li><a class="dropdown-toggle collapsed ntgBootstrap data-target_#trbuscador aria-expanded_true data-toggle_collapse" href="javascript:void(0);">
 <span class="fa fa-search"></span><span class="fa fa-times-circle-o"></span><span class="sr-only">Buscar</span></a>
 <div class="collapse panel panel-body" id="trbuscador">
 <!-- Buscador -->
 <div class="input-group input-group-lg buscador">
 <input class="form-control placefÂ¿holder-in-title" id="keywords" name="keywords" size="30" title="Buscar" type="text" value=""/>
 <script type="text/javascript">
               var src="https://www.spensiones.cl